In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from typing import Dict, Tuple
from langchain_core.chat_history import InMemoryChatMessageHistory

In [3]:
# 1. 모델 설정
llm = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature = 0,
)

# 2. 도구 생성
from schedule_toolkit import ScheduleToolkit
tools = ScheduleToolkit().get_tools()

# 3. 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 스케쥴을 관리하는 AI 비서야. 주어지는 도구를 잘 활용해서 사용자의 요청에 답하도록 해"),
    ("placeholder", "{history}"),
    ("user", "{question}"),
    ("placeholder", "{agent_scratchpad}")
])

# 4. 단일 에이전트 생성
agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# 5. excutor 설정
executor = AgentExecutor(
    agent = agent,
    tools=tools,
    verbose=True
)

In [4]:
# 1. 대화 내용 저장소 만들기
stores : Dict[str, InMemoryChatMessageHistory] = {}
def get_store(session_id: str):
    print(f"[대화 세션ID]: {session_id}")
    if session_id not in stores:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        stores[session_id] = InMemoryChatMessageHistory()
    return stores[session_id]  # 해당 세션 ID에 대한 세션 기록 반환

# 2. 히스토리랑 래핑
agent_history = RunnableWithMessageHistory(
    executor,
    lambda sid: get_store(sid),
    input_messages_key="question",
    history_messages_key="history"
)

In [5]:
cfg = {"configurable" : {"session_id" : "user-123"}}

In [6]:
agent_history.invoke({"question" : "오늘의 스케쥴에 맛있는 점심 먹기를 추가해줘"}, config=cfg)

[대화 세션ID]: user-123


> Entering new AgentExecutor chain...

Invoking: `add_todo` with `{'item': '맛있는 점심 먹기'}`


맛있는 점심 먹기이 스케쥴에 등록 되었습니다오늘의 스케쥴에 '맛있는 점심 먹기'를 추가했습니다.

> Finished chain.


{'question': '오늘의 스케쥴에 맛있는 점심 먹기를 추가해줘',
 'history': [],
 'output': "오늘의 스케쥴에 '맛있는 점심 먹기'를 추가했습니다."}

In [7]:
agent_history.invoke({"question" : "클라이언트 미팅 추가해줘"}, config=cfg)

[대화 세션ID]: user-123


> Entering new AgentExecutor chain...

Invoking: `add_todo` with `{'item': '클라이언트 미팅'}`


클라이언트 미팅이 스케쥴에 등록 되었습니다오늘의 스케쥴에 '클라이언트 미팅'을 추가했습니다.

> Finished chain.


{'question': '클라이언트 미팅 추가해줘',
 'history': [HumanMessage(content='오늘의 스케쥴에 맛있는 점심 먹기를 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '맛있는 점심 먹기'를 추가했습니다.", additional_kwargs={}, response_metadata={})],
 'output': "오늘의 스케쥴에 '클라이언트 미팅'을 추가했습니다."}

In [8]:
agent_history.invoke({"question" : "캍퇴하기 추가해줘"}, config=cfg)

[대화 세션ID]: user-123


> Entering new AgentExecutor chain...

Invoking: `add_todo` with `{'item': '캍퇴하기'}`


캍퇴하기이 스케쥴에 등록 되었습니다오늘의 스케쥴에 '캍퇴하기'를 추가했습니다.

> Finished chain.


{'question': '캍퇴하기 추가해줘',
 'history': [HumanMessage(content='오늘의 스케쥴에 맛있는 점심 먹기를 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '맛있는 점심 먹기'를 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='클라이언트 미팅 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '클라이언트 미팅'을 추가했습니다.", additional_kwargs={}, response_metadata={})],
 'output': "오늘의 스케쥴에 '캍퇴하기'를 추가했습니다."}

In [9]:
agent_history.invoke({"question" : "오늘의 스케쥴을 알려줘"}, config=cfg)

[대화 세션ID]: user-123


> Entering new AgentExecutor chain...

Invoking: `get_todo` with `{}`


할일 목록은 : 맛있는 점심 먹기
클라이언트 미팅
캍퇴하기오늘의 스케쥴은 다음과 같습니다:
1. 맛있는 점심 먹기
2. 클라이언트 미팅
3. 캍퇴하기

더 추가하거나 수정할 일정이 있나요?

> Finished chain.


{'question': '오늘의 스케쥴을 알려줘',
 'history': [HumanMessage(content='오늘의 스케쥴에 맛있는 점심 먹기를 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '맛있는 점심 먹기'를 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='클라이언트 미팅 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '클라이언트 미팅'을 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='캍퇴하기 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '캍퇴하기'를 추가했습니다.", additional_kwargs={}, response_metadata={})],
 'output': '오늘의 스케쥴은 다음과 같습니다:\n1. 맛있는 점심 먹기\n2. 클라이언트 미팅\n3. 캍퇴하기\n\n더 추가하거나 수정할 일정이 있나요?'}

In [10]:
agent_history.invoke({"question" : "점심 먹기는 완료했어!"}, config=cfg)

[대화 세션ID]: user-123


> Entering new AgentExecutor chain...

Invoking: `delete_todo` with `{'item': '맛있는 점심 먹기'}`


맛있는 점심 먹기이 스케쥴에 삭제 되었습니다'맛있는 점심 먹기' 일정을 완료하셨으니 스케쥴에서 삭제했습니다. 다른 일정도 관리해드릴까요?

> Finished chain.


{'question': '점심 먹기는 완료했어!',
 'history': [HumanMessage(content='오늘의 스케쥴에 맛있는 점심 먹기를 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '맛있는 점심 먹기'를 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='클라이언트 미팅 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '클라이언트 미팅'을 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='캍퇴하기 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '캍퇴하기'를 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='오늘의 스케쥴을 알려줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='오늘의 스케쥴은 다음과 같습니다:\n1. 맛있는 점심 먹기\n2. 클라이언트 미팅\n3. 캍퇴하기\n\n더 추가하거나 수정할 일정이 있나요?', additional_kwargs={}, response_metadata={})],
 'output': "'맛있는 점심 먹기' 일정을 완료하셨으니 스케쥴에서 삭제했습니다. 다른 일정도 관리해드릴까요?"}

In [11]:
agent_history.invoke({"question" : "칼퇴하기는 실패했어...취소해줘"}, config=cfg)

[대화 세션ID]: user-123


> Entering new AgentExecutor chain...

Invoking: `delete_todo` with `{'item': '캍퇴하기'}`


캍퇴하기이 스케쥴에 삭제 되었습니다'캍퇴하기' 일정을 스케쥴에서 취소(삭제)했습니다. 다른 일정도 도와드릴까요?

> Finished chain.


{'question': '칼퇴하기는 실패했어...취소해줘',
 'history': [HumanMessage(content='오늘의 스케쥴에 맛있는 점심 먹기를 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '맛있는 점심 먹기'를 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='클라이언트 미팅 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '클라이언트 미팅'을 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='캍퇴하기 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '캍퇴하기'를 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='오늘의 스케쥴을 알려줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='오늘의 스케쥴은 다음과 같습니다:\n1. 맛있는 점심 먹기\n2. 클라이언트 미팅\n3. 캍퇴하기\n\n더 추가하거나 수정할 일정이 있나요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='점심 먹기는 완료했어!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="'맛있는 점심 먹기' 일정을 완료하셨으니 스케쥴에서 삭제했습니다. 다른 일정도 관리해드릴까요?", additional_kwa

In [12]:
agent_history.invoke({"question" : "오늘의 남은 스케쥴을 알려줘"}, config=cfg)

[대화 세션ID]: user-123


> Entering new AgentExecutor chain...

Invoking: `get_todo` with `{}`


할일 목록은 : 클라이언트 미팅오늘 남은 스케쥴은 '클라이언트 미팅'입니다. 다른 일정 관리가 필요하시면 말씀해 주세요.

> Finished chain.


{'question': '오늘의 남은 스케쥴을 알려줘',
 'history': [HumanMessage(content='오늘의 스케쥴에 맛있는 점심 먹기를 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '맛있는 점심 먹기'를 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='클라이언트 미팅 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '클라이언트 미팅'을 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='캍퇴하기 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '캍퇴하기'를 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='오늘의 스케쥴을 알려줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='오늘의 스케쥴은 다음과 같습니다:\n1. 맛있는 점심 먹기\n2. 클라이언트 미팅\n3. 캍퇴하기\n\n더 추가하거나 수정할 일정이 있나요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='점심 먹기는 완료했어!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="'맛있는 점심 먹기' 일정을 완료하셨으니 스케쥴에서 삭제했습니다. 다른 일정도 관리해드릴까요?", additional_kwarg

In [13]:
agent_history.invoke({"question" : "보고서 작성하기 스케쥴을 취소해줘"}, config=cfg)

[대화 세션ID]: user-123


> Entering new AgentExecutor chain...
오늘 스케쥴에 '보고서 작성하기' 일정이 없습니다. 다른 일정에 대해 도와드릴까요?

> Finished chain.


{'question': '보고서 작성하기 스케쥴을 취소해줘',
 'history': [HumanMessage(content='오늘의 스케쥴에 맛있는 점심 먹기를 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '맛있는 점심 먹기'를 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='클라이언트 미팅 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '클라이언트 미팅'을 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='캍퇴하기 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content="오늘의 스케쥴에 '캍퇴하기'를 추가했습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='오늘의 스케쥴을 알려줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='오늘의 스케쥴은 다음과 같습니다:\n1. 맛있는 점심 먹기\n2. 클라이언트 미팅\n3. 캍퇴하기\n\n더 추가하거나 수정할 일정이 있나요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='점심 먹기는 완료했어!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="'맛있는 점심 먹기' 일정을 완료하셨으니 스케쥴에서 삭제했습니다. 다른 일정도 관리해드릴까요?", additional_kw